In [ ]:
import commpy
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt

In [ ]:
# OFDM parameters
K = 64
CP = K//4
P = 32  
pilotValue = 3+3j
Modulation_type = 'QAM64'

# Channel parameters
channel_type ='random'
channel_coefs =np.array([1, 0.5j, 0.3j, 0.1j])
SNRdb = 25

In [ ]:
# Set up OFDM
allCarriers = np.arange(K)
pilotCarrier = allCarriers[::K//P]
pilotCarriers = np.hstack([pilotCarrier, np.array([allCarriers[-1]])])
P = P+1 
m_map = {"BPSK": 1, "QPSK": 2, "8PSK": 3, "QAM16": 4, "QAM64": 6}
mu = m_map[Modulation_type]
dataCarriers = np.delete(allCarriers, pilotCarriers)
payloadBits_per_OFDM = len(dataCarriers)*mu 

In [ ]:
def modulate(bits):
    QAM64 = commpy.QAMModem(64)
    symbol = QAM64.modulate(bits)
    return symbol

def demodulate(symbol):
    QAM64 = commpy.QAMModem(64)
    bits = QAM64.demodulate(symbol, demod_type='hard')
    return bits

def OFDM_symbol(QAM_payload):
    symbol = np.zeros(K, dtype=complex)  
    symbol[pilotCarriers] = pilotValue 
    symbol[dataCarriers] = QAM_payload
    return symbol

def IDFT(OFDM_data):
    return np.fft.ifft(np.fft.ifftshift(OFDM_data))

def addCP(OFDM_time):
    cp = OFDM_time[-CP:]
    return np.hstack([cp, OFDM_time])

def removeCP(signal):
    return signal[CP:(CP+K)]

def DFT(OFDM_RX):
    return np.fft.fftshift(np.fft.fft(OFDM_RX))

def channelEstimate(OFDM_demod):
    pilots = OFDM_demod[pilotCarriers]
    Hest_at_pilots = pilots / pilotValue
    
    Hest_abs = interpolate.interp1d(pilotCarriers, abs(
        Hest_at_pilots), kind='linear')(allCarriers)
    Hest_phase = interpolate.interp1d(pilotCarriers, np.angle(
        Hest_at_pilots), kind='linear')(allCarriers)
    Hest = Hest_abs * np.exp(1j*Hest_phase)
    return Hest

def equalize(OFDM_demod, Hest):
    return OFDM_demod / Hest

In [ ]:
# Channel effects
def add_noise(x_s, snrDB):
    data_pwr = np.mean(abs(x_s**2))
    noise_pwr = data_pwr/(10**(snrDB/10))
    noise = 1/np.sqrt(2) * (np.random.randn(len(x_s)) + 1j *
                            np.random.randn(len(x_s))) * np.sqrt(noise_pwr)
    return x_s + noise, noise_pwr

def channel(in_signal, SNRdb):
    out_signal = np.convolve(in_signal, channel_coefs)

    return out_signal

In [ ]:
def fuzzing(in_signal, c1_raw, c1_rot, c2_raw, c2_rot):
    if c1_rot == 0:
        c1 = c1_raw/128
    else:
        c1 = complex(0,c1_raw/128)

    if c2_rot == 0:
        c2 = c2_raw/128
    else:
        c2 = complex(0,c2_raw/128)

    fuzzing = np.array([1,c1,c2])
    out_signal = np.convolve(in_signal, fuzzing)
    return out_signal

In [ ]:
def csi_plot(array, title, legend):
    fig, axs = plt.subplots(2, figsize=(12.8,9.6))
    fig.suptitle('CSI ' + title)
    x_amp = np.arange(K)
    x_pha = np.arange(K)
    for CSI in array:
        axs[0].plot(x_amp, np.abs(CSI))
        axs[1].plot(x_pha, np.angle(CSI,deg=False))
    plt.legend(legend)
    plt.show()

In [ ]:
# Simulation control parameters
enable_fuzzing = True
enable_channel = False
enable_noise = False

In [ ]:
### Transmit ###

# Generate some random bits to be send
bits = np.random.binomial(n=1, p=0.5, size=(payloadBits_per_OFDM, ))
    
# Modulate the bits into OFDM symbols
QAM_s = modulate(bits)
OFDM_data = OFDM_symbol(QAM_s)
    
# Tranform symbols to time domain and add cyclic prefix
OFDM_time = IDFT(OFDM_data)
OFDM_withCP = addCP(OFDM_time)


if enable_fuzzing == True:
    # Apply fuzzing
    OFDM_fuzzed = fuzzing(OFDM_withCP,-20,0,-20,0)
    OFDM_TX = OFDM_fuzzed
else:
    OFDM_TX = OFDM_withCP

In [ ]:
### Channel effects ###
if enable_channel == True:
    OFDM_RX = channel(OFDM_TX, SNRdb)
else:
    OFDM_RX = OFDM_TX

if enable_noise == True:
    OFDM_RX = add_noise(OFDM_RX,SNRdb)[0]

In [ ]:

### Receiver ###

# Remove cp and transform to frequency domain
OFDM_RX_noCP = removeCP(OFDM_RX)
OFDM_demod = DFT(OFDM_RX_noCP)
    
# Perform channel state estimation
CSI = channelEstimate(OFDM_demod)
      
# Calculate the defuzzed_csi, to be used for equalisation
unfuzzed_csi = channelEstimate(DFT(removeCP(channel(OFDM_withCP, SNRdb))))
fft = np.fft.fftshift(np.fft.fft([1,-20/128,-20/128],64))

if enable_fuzzing == True:
    defuzzed_csi = CSI/fft
else:
    defuzzed_csi = CSI

# Equalize and retrieve data
equalized_Hest = equalize(OFDM_demod, defuzzed_csi)
QAM_est = equalized_Hest[dataCarriers]
    
# Retrieve the original bits
bits_est = demodulate(QAM_est)

In [ ]:
# Plot the CSI, and the defuzzed CSI
csi_plot([CSI,fft], "comparison",["measured","predicted"])
csi_plot([defuzzed_csi,unfuzzed_csi],"defuzzed",["defuzzed","unfuzzed"])